In [10]:
# do all necessary imports for this chapter
import numpy as np
from scipy.linalg import rq, qr, svd
from scipy.sparse.linalg import eigs, LinearOperator
from ncon import ncon

# Tangent-space methods for uniform matrix product states

https://arxiv.org/abs/1810.07006

## 1. Matrix product states in the thermodynamic limit
### 1.1 Normalisation
We start by considering a uniform MPS in the thermodynamic limit, which is defined by 
$$ |\Psi(A)> = \sum_i^d \nu_L^\dagger \left[ \prod_{m\in Z} A_i \right]\nu_R |i>. $$

Here, $A_i$ are complex matrices of size $D \times D$, for every entry of the index $d$. This allows for the interpretation of the object $A$ as a three-legged tensor of dimensions $D\times d \times D$, where we will refer to $D$ as the bond dimension and $d$ as the physical dimension. With this object and the diagrammatic language of tensor networks, we can represent the state as

![image.png](img/MPSstate.png)

Thus, we initialise and represent a uniform MPS state as follows:

In [11]:
def createMPS(D, d):
    """
    Returns a random complex MPS tensor.
    
        Parameters
        ----------
        D : int
            Bond dimension for MPS.
        d : int
            Physical dimension for MPS.
            
        Returns
        -------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
    """
    
    A = np.random.rand(D, d, D) + 1j*np.random.rand(D, d, D)
    
    return A

In [12]:
d, D = 3, 5
A = createMPS(D, d)

assert A.shape == (D, d, D), "Generated MPS tensor has incorrect shape."
assert A.dtype == 'complex128', "MPS tensor should have complex values"

One of the most relevant objects in all our calculations will be the transfer matrix, defined as

![image.png](img/transferMatrix.png)

where we will be using the convention of ordering the legs as
1. top left
2. bottom left
3. top right
4. bottom right

The transfer matrix can be shown to be a completely positive map, such that the leading eigenvalue is a positive number, which we conveniently rescale to unity for a proper normalization of the state in the thermodynamic limit. This means solving the (left and right) eigenvalue equation:
![image.png](img/fixedPoints.png)

Additionally, we may fix the normalisation of the eigenvectors by requiring that their trace is equal to one:
![image.png](img/traceNorm.png)

If we now require that there is no influence of the boundary on the bulk, we require the overlap of the boundary vectors with the fixed points to equal unity. In this case, we will have properly normalised the MPS:
![image.png](img/MPSnormalised.png)

In [13]:
def createTransfermatrix(A):
    """
    Form the transfermatrix of an MPS.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        E : np.array(D, D, D, D)
            Transfermatrix with 4 legs,
            ordered topLeft-bottomLeft-topRight-bottomRight.
    """
    
    E = ncon((A, np.conj(A)), ([-1, 1, -3], [-2, 1, -4]))
    
    return E

In [14]:
def normaliseMPS(A):
    """
    Normalise an MPS tensor.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        Anew : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) matrix.
    """

    D = A.shape[0]
    E = createTransfermatrix(A)

    # calculate eigenvalue
    norm = eigs(np.reshape(E, (D ** 2, D ** 2)), k=1, which='LM', return_eigenvectors=False)
    Anew = A / np.sqrt(norm)
                 
    return Anew

In [15]:
def leftFixedPoint(A):
    """
    Find left fixed point.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        l : np.array(D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) matrix.
    """
    
    D = A.shape[0]
    E = createTransfermatrix(A)
    
    # find fixed point
    _, l = eigs(E.reshape((D ** 2, D ** 2)).T, k=1, which='LM')
    
    # fix shape/order l
    l = l.reshape((D, D)).T
    
    # make left fixed point hermitian and positive semidefinite explicitly
    l /= (np.trace(l) / np.abs(np.trace(l)))# remove possible phase
    l = (l + np.conj(l).T) / 2 # force hermitian
    l *= np.sign(np.trace(l)) # force positive semidefinite

    return l

In [16]:
def rightFixedPoint(A):
    """
    Find right fixed point.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        r : np.array(D, D)
            right fixed point with 2 legs,
            ordered top-bottom.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) matrix.
    """
    
    D = A.shape[0]
    E = createTransfermatrix(A)
    
    # find fixed point
    _, r = eigs(E.reshape((D ** 2, D ** 2)), k=1, which='LM')
    
    # fix shape r
    r = r.reshape((D, D))
    
    # make right fixed point hermitian and positive semidefinite explicitly
    r /= (np.trace(r) / np.abs(np.trace(r)))# remove possible phase
    r = (r + np.conj(r).T) / 2 # force hermitian
    r *= np.sign(np.trace(r)) # force positive semidefinite
    
    return r

In [17]:
def fixedPoints(A):
    """
    Find normalised fixed points.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        l : np.array(D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
        r : np.array(D, D)
            right fixed point with 2 legs,
            ordered top-bottom.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) matrix
    """
    
    # find fixed points
    l, r = leftFixedPoint(A), rightFixedPoint(A)
    
    # calculate trace
    trace = np.trace(l@r)
    
    return l / trace, r

In [18]:
A = normaliseMPS(A)
l, r = fixedPoints(A)

assert np.allclose(l, np.conj(l).T, 1e-12), "left fixed point should be hermitian!"
assert np.allclose(r, np.conj(r).T, 1e-12), "right fixed point should be hermitian!"

assert np.allclose(l, ncon((A, l, np.conj(A)), ([1, 2, -2], [3, 1], [3, 2, -1])), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, ncon((A, r, np.conj(A)), ([-1, 2, 1], [1, 3], [-2, 2, 3])), 1e-12), "r should be a right fixed point!"
assert np.abs(ncon((l, r), ([1, 2], [2, 1])) - 1) < 1e-12, "Left and right fixed points should be trace normalised!"

### 1.2 Gauge fixing
While a single $A$ corresponds to a unique state $|\Psi(A)>$, the converse is not true, as different tensors may give rise to the same state. This is easily seen by noting that the gauge transform
![image.png](img/gaugeTransform.png)
leaves the physical state invariant. We may use this freedom in parametrization to impose canonical forms on the MPS tensor $A$.

We start by considering the left-orthonormal form $A_L$ of an MPS, which has the property that
![image.png](img/leftOrthonormal.png).

We can find the matrix $L$ that brings $A$ into this form by decomposing the fixed point $l$ as $l = L^\dagger L$, such that
![image.png](img/leftOrthonormal2.png)

Furthermore, there is still room for unitary gauge transformations such that we can also bring the right fixed point in diagonal form. Similarly, we can consider the right-orthonormal form $A_R$, where we have
![image.png](img/rightOrthonormal.png)

In [19]:
"""As transfer matrix is positive definite, fixed points are also positive definite, thus allowing for a
square root algorithm of the eigenvalues. Note that taking square roots is typically less favourable,
as this increases the error"""

def leftOrthonormalise(A, l=None):
    """
    Transform A to left-orthonormal gauge.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        L : np.array(D, D)
            left gauge with 2 legs,
            ordered left-right.
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) matrix
    """
    
    # find left fixed point
    if l is None:
        l = leftFixedPoint(A)
    
    # decompose l = Ldag * L
    S, U = np.linalg.eig(l)
    L = U @ np.diag(np.sqrt(S)) @ np.conj(U).T
    
    # apply gauge L to A
    Al = ncon((L, A, np.linalg.inv(L)), ([-1, 1], [1, -2, 2], [2, -3]))

    return L, Al

In [20]:
def rightOrthonormalise(A, r=None):
    """
    Transform A to right-orthonormal gauge.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        R : np.array(D, D)
            right gauge with 2 legs,
            ordered left-right.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalising (D ** 2, D ** 2) dmatrix
    """
    
    # find right fixed point
    if r is None:
        r = rightFixedPoint(A)
    
    # decompose r = Rdag * R
    S, U = np.linalg.eig(r)
    R = U @ np.diag(np.sqrt(S)) @ np.conj(U).T
    
    # apply gauge R to A
    Ar = ncon((np.linalg.inv(R), A, R), ([-1, 1], [1, -2, 2], [2, -3]))

    return R, Ar

In [21]:
L, Al = leftOrthonormalise(A, l)
R, Ar = rightOrthonormalise(A, r)

assert np.allclose(R @ np.conj(R).T, r), "Right gauge doesn't square to r"
assert np.allclose(np.conj(L).T @ L, l), "Left gauge doesn't sqaure to l"
assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"

Finally, there is the notion of a mixed gauge for the uniform MPS by choosing a 'center site', bringing the left tensors in left-orthonormal form and the right tensors in right-orthonormal form, and redefining the center tensor as follows:
![image.png](img/mixedGauge.png)

The mixed gauge has an intuitive interpretation. Defining $C = LR$, this implements the gauge transform that maps the left-orthonormal tensor to the right-orthonormal one, hence defining the center-site tensor $A_C$:
![image.png](img/mixedGauge2.png)
The above is called the mixed gauge condition and allows us to freely move the center tensor $A_C$ around in the MPS.


Finally we may perform an SVD of $C = USV^\dagger$, and taking up $U$ and $V^\dagger$ in the definition of $A_L$ and $A_R$, such that we are left with a diagonal $C$ on the virtual bonds.
![image.png](img/mixedGauge3.png)

In fact, this means that we can write down a Schmidt decomposition of the state across an arbitrary bond in the chain, and the diagonal elements $C_l$ are exactly the Schmidt numbers of any bipartition of the MPS. 
![image.png](img/SchmidtDecomp.png)
Hence, we can calculate the bipartite entanglement entropy by 
$$ S = -\sum_l C_l^2 \log(C_l^2) $$


In [22]:
def mixedCanonical(A):
    """
    Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ac : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalisation of (D ** 2, D ** 2) matrix
    """

    D = A.shape[0]

    # Compute left and right orthonormal forms
    L, Al = leftOrthonormalise(A)
    R, Ar = rightOrthonormalise(A)
    
    # center matrix C is matrix multiplication of L and R
    C = L @ R
    
    # singular value decomposition to diagonalise C
    U, S, Vdag = svd(C)
    C = np.diag(S)

    # absorb corresponding unitaries in Al and Ar
    Al = ncon((np.conj(U).T, Al, U), ([-1, 1], [1, -2, 2], [2, -3]))
    Ar = ncon((Vdag, Ar, np.conj(Vdag).T), ([-1, 1], [1, -2, 2], [2, -3]))

    # normalise center matrix
    norm = np.trace(C @ np.conj(C).T)
    C /= np.sqrt(norm)

    # compute center MPS tensor
    Ac = ncon((Al, C), ([-1, -2, 1], [1, -3]))

    return Al, Ac, Ar, C

In [23]:
def entanglementSpectrum(A):
    """
    Calculate the entanglement spectrum of an MPS.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        S : np.array (D,)
            Singular values of center matrix,
            representing the entanglement spectrum
        entropy : float
            Entanglement entropy across a leg.
    """
    
    # go to mixed gauge
    _, _, _, C = mixedCanonical(A)

    # calculate entropy
    S = np.diag(C)
    entropy = -np.sum(S**2 * np.log(S ** 2))
    
    return S, entropy

In [24]:
# check that gauging is still correct
Al, Ac, Ar, C = mixedCanonical(A)
S, entropy = entanglementSpectrum(A)

assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"
LHS = ncon((Al, C), ([-1, -2, 1], [1, -3]))
RHS = ncon((C, Ar), ([-1, 1], [1, -2, -3]))
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

### 1.3 Truncation of a uniform MPS
The mixed canonical form is not only useful for finding the entanglement entropy, it also allows for a way to truncate MPS states efficiently. This is done by truncating the Schmidt decomposition, such that the new MPS has a reduced bond dimension for that ond. This truncation can be shown to be optimal in the sense that the norm between the original and the truncated MPS is minimised. The truncated MPS in the mixed gauge is thus:
![image.png](img/truncMPS.png)

We mention that this is a local optimization, in the sense that it maximizes the local overlap, however not the global overlap. This would require a variational optimization of the following cost function:
$$ || ~|\Psi(A)> - |\Psi(\tilde{A})> ||^2 $$
We postpone the detailed discussion hereof until later.

In [25]:
def truncateMPS(A, Dtrunc):
    """
    Truncate an MPS to a lower bond dimension.
    
        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        Dtrunc : int
            lower bond dimension
            
        Returns
        -------
        AlTilde : np.array(Dtrunc, d, Dtrunc)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        AcTilde : np.array(Dtrunc, d, Dtrunc)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        ArTilde : np.array(Dtrunc, d, Dtrunc)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        CTilde : np.array(Dtrunc, Dtrunc)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.
    """
    
    Al, Ac, Ar, C = mixedCanonical(A)
    
    # perform SVD and truncate:
    U, S, Vdag = svd(C)
    U = U[:,:Dtrunc]
    Vdag = Vdag[:Dtrunc,:]
    S = S[:Dtrunc]
    
    # reabsorb unitaries
    AlTilde = ncon((np.conj(U).T, Al, U), ([-1, 1], [1, -2, 2], [2, -3]))
    ArTilde = ncon((Vdag, Ar, np.conj(Vdag).T), ([-1, 1], [1, -2, 2], [2, -3]))
    CTilde = np.diag(S)
    
    # normalise center matrix
    norm = np.trace(CTilde @ np.conj(CTilde).T)
    CTilde /= np.sqrt(norm)

    # compute center MPS tensor
    AcTilde = ncon((AlTilde, CTilde), ([-1, -2, 1], [1, -3]))
    
    return AlTilde, AcTilde, ArTilde, CTilde

In [26]:
AlTilde, AcTilde, ArTilde, CTilde = truncateMPS(A, 3)
assert AlTilde.shape[0] == 3 and AlTilde.shape[2] == 3, "Something went wrong in truncating the MPS"

### 1.4 Algorithms for finding canonical forms
One of the great things of MPS is that they provide efficient approximations of physical states, and for gapped systems they are expected to be exact in the limit $D \rightarrow \infty$. The idea is that if we increase the bond dimension enough, we can get to arbitrary precision. However, increasing the bond dimension comes at a numerical cost, as the MPS algorithms scale in $D$. It is possible to ensure that the complexity of all MPS algorithms scales as $O(D^3)$, however this means we need to be a little smarter with the previously encountered algorithms.

As a first step, we can refrain from explicitly building the matrices that are used in the eigenvalue solvers, but this will easily extend to all following algorithms. We can circumvent explicitly building the matrix by implementing a function that corresponds to the action of the operator, usually called a handle, and passing this to the eigenvalue solver.

For example, a better way of fixing the normalisation of an MPS tensor, as well as determining left and right fixed points is by implementing the handles and using the optimal contraction sequences:

In [ ]:
def normaliseMPS(A):
    """
    Normalise an MPS tensor.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        Anew : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    D = A.shape[0]

    # calculate transfer matrix handle and cast to LinearOperator
    handleERight = lambda v: np.reshape(ncon((A, np.conj(A), v.reshape((D,D))), ([-1, 2, 1], [-2, 2, 3], [1, 3])),
                                        D ** 2)
    E = LinearOperator((D ** 2, D ** 2), matvec=handleERight)

    # calculate eigenvalue
    lam = eigs(E, k=1, which='LM', return_eigenvectors=False)

    Anew = A / np.sqrt(lam)

    return Anew

In [ ]:
def leftFixedPoint(A):
    """
    Find left fixed point.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        l : np.array(D, D)
            left fixed point with 2 legs,
            ordered bottom-top.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    D = A.shape[0]

    # calculate transfer matrix handle and cast to LinearOperator
    handleELeft = lambda v: np.reshape(ncon((A, np.conj(A), v.reshape((D, D))), ([1, 2, -2], [3, 2, -1], [3, 1])), D ** 2)
    E = LinearOperator((D ** 2, D ** 2), matvec=handleELeft)

    # calculate fixed point
    _, l = eigs(E, k=1, which='LM')
    
    # reshape to matrix
    l = l.reshape((D, D))
    
    # make left fixed point hermitian explicitly
    l /= (np.trace(l) / np.abs(np.trace(l)))# remove possible phase
    l = (l + np.conj(l).T) / 2 # force hermitian
    l *= np.sign(np.trace(l)) # force positive semidefinite

    return l

In [ ]:
def rightFixedPoint(A):
    """
    Find right fixed point.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        r : np.array(D, D)
            right fixed point with 2 legs,
            ordered top-bottom.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    D = A.shape[0]

    # calculate transfer matrix handle and cast to LinearOperator
    handleERight = lambda v: np.reshape(ncon((A, np.conj(A), v.reshape((D,D))), ([-1, 2, 1], [-2, 2, 3], [1, 3])), D ** 2)
    E = LinearOperator((D ** 2, D ** 2), matvec=handleERight)

    # calculate fixed point
    _, r = eigs(E, k=1, which='LM')
    
    # reshape to matrix
    r = r.reshape((D, D))
    
    # make right fixed point hermitian explicitly
    r /= (np.trace(r) / np.abs(np.trace(r)))# remove possible phase
    r = (r + np.conj(r).T) / 2 # force hermitian
    r *= np.sign(np.trace(r)) # force positive semidefinite
    
    return r

In [ ]:
def fixedPoints(A):
    """
    Find normalised fixed points.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        l : np.array(D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
        r : np.array(D, D)
            right fixed point with 2 legs,
            ordered top-bottom.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    # find fixed points
    l, r = leftFixedPoint(A), rightFixedPoint(A)

    # calculate trace
    trace = np.trace(l@r)

    return l / trace, r

In [ ]:
A = normaliseMPS(A)
l, r = fixedPoints(A)

assert np.allclose(l, np.conj(l).T, 1e-12), "left fixed point should be hermitian!"
assert np.allclose(r, np.conj(r).T, 1e-12), "right fixed point should be hermitian!"

assert np.allclose(l, ncon((A, l, np.conj(A)), ([1, 2, -2], [3, 1], [3, 2, -1])), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, ncon((A, r, np.conj(A)), ([-1, 2, 1], [1, 3], [-2, 2, 3])), 1e-12), "r should be a right fixed point!"
assert abs(ncon((l, r), ([1, 2], [2, 1])) - 1) < 1e-12, "Left and right fixed points should be trace normalised!"

Furthermore, taking the square root of such a fixed point to find the left and right gauge is also a way to introduce larger errors than neccesary. This can be circumvented by using an iterative scheme that makes use of a unique decomposition, such as a QR or polar decomposition. Note however that for a QR decomposition to be unique we need the diagonal elements of R to have a positive sign.

In [ ]:
def rqPos(A):
    """
    Do a RQ decomposition with positive diagonal elements for R.
    
        Parameters
        ----------
        A : np.array(M, N)
            Matrix to decompose.
            
        Returns
        -------
        R : np.array(M, M)
            Upper triangular matrix,
            positive diagonal elements.
        Q : np.array(M, N)
            Orthogonal matrix.
            
        Complexity
        ----------
        ~O(max(M, N) ** 3) algorithm.
    """

    
    M, N = A.shape
    
    # LQ decomposition: scipy conventions: Q.shape = (N, N), L.shape = (M, N)
    R, Q = rq(A)

    # Throw out zeros under diagonal: Q.shape = (M, N), L.shape = (M, M)
    Q = Q[-M:, :]
    R = R[:, -M:]

    # Extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(R)))
    Q = np.dot(diagSigns, Q)
    R = np.dot(R, diagSigns)

    return R, Q


def rightOrthonormalise(A, R0=None, tol=1e-14, maxIter=1e5):
    """
    Transform A to right-orthonormal gauge.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        R0 : np.array(D, D), optional
            Right gauge matrix,
            initial guess.
        tol : float, optional
            convergence criterium,
            norm(R - Rnew) < tol.
        maxIter : int
            maximum amount of iterations.

        Returns
        -------
        R : np.array(D, D)
            right gauge with 2 legs,
            ordered left-right.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right-orthonormal
    """

    D = A.shape[0]
    d = A.shape[1]
    i = 1

    # Random guess for R0 if none specified
    if R0 is None:
        R0 = np.random.rand(D, D)

    # Normalise R0
    R0 = R0 / np.linalg.norm(R0)

    # Initialise loop
    R, Ar = rqPos(np.reshape(ncon((A, R0), ([-1, -2, 1], [1, -3])), (D, D * d)))
    R = R / np.linalg.norm(R)
    convergence = np.linalg.norm(R - R0)

    # Decompose A*R until R converges
    while convergence > tol:
        # calculate AR and decompose
        Rnew, Ar = rqPos(np.reshape(ncon((A, R), ([-1, -2, 1], [1, -3])), (D, D * d)))

        # normalise new R
        Rnew = Rnew / np.linalg.norm(Rnew)

        # calculate convergence criterium
        convergence = np.linalg.norm(Rnew - R)
        R = Rnew

        # check if iterations exceeds maxIter
        if i > maxIter:
            print("Warning, right decomposition has not converged ", convergence)
            break
        i += 1

    return R, Ar.reshape((D, d, D))

In [ ]:
def qrPos(A):
    """
    Do a QR decomposition with positive diagonal elements for R.
    
        Parameters
        ----------
        A : np.array(M, N)
            Matrix to decompose.
            
        Returns
        -------
        Q : np.array(M, N)
            Orthogonal matrix.
        R : np.array(N, N)
            Upper triangular matrix,
            positive diagonal elements.
            
        Complexity
        ----------
        ~O(max(M, N) ** 3) algorithm.
    """
    
    M, N = A.shape
    
    # QR decomposition, scipy conventions: Q.shape = (M, M), R.shape = (M, N)
    Q, R = qr(A)
    
    # Throw out zeros under diagonal: Q.shape = (M, N), R.shape = (N, N)
    Q = Q[:, :N]
    R = R[:N, :]

    # extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(R)))
    Q = np.dot(Q, diagSigns)
    R = np.dot(diagSigns, R)
    
    return Q, R


def leftOrthonormalise(A, L0=None, tol=1e-14, maxIter=1e5):
    """
    Transform A to left-orthonormal gauge.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        L0 : np.array(D, D), optional
            Left gauge matrix,
            initial guess.
        tol : float, optional
            convergence criterium,
            norm(R - Rnew) < tol.
        maxIter : int
            maximum amount of iterations.

        Returns
        -------
        L : np.array(D, D)
            left gauge with 2 legs,
            ordered left-right.
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left-orthonormal
    """

    D = A.shape[0]
    d = A.shape[1]
    i = 1

    # Random guess for L0 if none specified
    if L0 is None:
        L0 = np.random.rand(D, D)

    # Normalise L0
    L0 = L0 / np.linalg.norm(L0)

    # Initialise loop
    Al, L = qrPos(np.reshape(ncon((L0, A), ([-1, 1], [1, -2, -3])), (D * d, D)))
    L = L / np.linalg.norm(L)
    convergence = np.linalg.norm(L - L0)

    # Decompose L*A until L converges
    while convergence > tol:
        # calculate LA and decompose
        Al, Lnew = qrPos(np.reshape(ncon((L, A), ([-1, 1], [1, -2, -3])), (D * d, D)))

        # normalise new L
        Lnew = Lnew / np.linalg.norm(Lnew)

        # calculate convergence criterium
        convergence = np.linalg.norm(Lnew - L)
        L = Lnew

        # check if iterations exceeds maxIter
        if i > maxIter:
            print("Warning, left decomposition has not converged ", convergence)
            break
        i += 1

    return L, Al.reshape((D, d, D))

In [ ]:
def mixedCanonical(A, L0=None, R0=None, tol=1e-14, maxIter=1e5):
    """
    Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.

        Parameters
        ----------
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ac : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.

        Complexity
        ----------
        O(D ** 3) algorithm.
    """

    D = A.shape[0]

    # Random guess for  L0 if none specified
    if L0 is None:
        L0 = np.random.rand(D, D)

    # Random guess for  R0 if none specified
    if R0 is None:
        R0 = np.random.rand(D, D)

    # Compute left and right orthonormal forms
    L, Al = leftOrthonormalise(A, L0, tol, maxIter)
    R, Ar = rightOrthonormalise(A, R0, tol, maxIter)

    # center matrix C is matrix multiplication of L and R
    C = L @ R

    # singular value decomposition to diagonalise C
    U, S, Vdag = svd(C)
    C = np.diag(S)

    # absorb corresponding unitaries in Al and Ar
    Al = ncon((np.conj(U).T, Al, U), ([-1, 1], [1, -2, 2], [2, -3]))
    Ar = ncon((Vdag, Ar, np.conj(Vdag).T), ([-1, 1], [1, -2, 2], [2, -3]))

    # normalise center matrix
    norm = np.trace(C @ np.conj(C).T)
    C /= np.sqrt(norm)

    # compute center MPS tensor
    Ac = ncon((Al, C), ([-1, -2, 1], [1, -3]))

    return Al, Ac, Ar, C

In [ ]:
Al, Ac, Ar, C = mixedCanonical(A)
S, entropy = entanglementSpectrum(A)

assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"
LHS = ncon((Al, C), ([-1, -2, 1], [1, -3]))
RHS = ncon((C, Ar), ([-1, 1], [1, -2, -3]))
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

#### Note:
For the remainder of this tutorial, keep in mind that you should always aim to reduce the complexity of your algorithm to $O(D^3)$ in order to keep the computational cost tractable.

### 1.5 Computing expectation values 
Having described the states in different gauges, we want to use these to determine the expectation values of extensive operators:
$$ O = \frac{1}{Z} \sum_{n\in Z} O_n $$

If we work with properly normalized MPS, the expectation value per site of a one-body operator (such as an order parameter e.g. $X_i$) is then found by considering the following contraction:
![image.png](img/expValue.png)

If we use the uniform gauge, we can use the fixed points of the transfer matrix to collapse everything on the left and right, such that we are left with the contraction:
![image.png](img/expValue2.png)

However, in the mixed gauge, we can locate the center site where the operator is acting, and then contract everything to the left and right to the identity, such that we find
![image.png](img/expValue3.png)

In [ ]:
def expVal1Uniform(O, A, l=None, r=None):
    """
    Calculate the expectation value of a 1-site operator in uniform gauge.

        Parameters
        ----------
        O : np.array(d, d)
            single-site operator.
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    # calculate fixed points if not given
    if l is None or r is None:
        l, r = fixedPoints(A)

    # contract expectation value network
    o = ncon((l, r, A, np.conj(A), O), ([4, 1], [3, 6], [1, 2, 3], [4, 5, 6], [2, 5]))

    return o

In [ ]:
def expVal1Mixed(O, Ac):
    """
    Calculate the expectation value of a 1-site operator in mixed gauge.

        Parameters
        ----------
        O : np.array(d, d)
            single-site operator.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauged.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    # contract expectation value network
    o = ncon((Ac, np.conj(Ac), O), ([1, 2, 3], [1, 4, 3], [2, 4]), order=[2, 1, 3, 4])

    return o

In [ ]:
O = np.random.rand(d,d) + 1.0j * np.random.rand(d,d)
A = createMPS(D, d)
A = normaliseMPS(A)
Al, Ac, Ar, C = mixedCanonical(A)
expVal = expVal1Uniform(O, A)
expValMix = expVal1Mixed(O, Ac)
diff = abs(expVal - expValMix)
assert diff < 1e-14, "different gauges give different values?"

This procedure generalises easily to operators that act on multiple sites. In particular, the Hamiltonian $h$ can be evaluated as
![image.png](img/hamExpVal.png)

In [ ]:
def expVal2Uniform(O, A, l=None, r=None):
    """
    Calculate the expectation value of a 2-site operator in uniform gauge.

        Parameters
        ----------
        O : np.array(d, d, d, d)
            two-site operator,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array(D, D), optional
            left fixed point of transfermatrix,
            normalised.
        r : np.array(D, D), optional
            right fixed point of transfermatrix,
            normalised.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    # calculate fixed points if not given
    if l is None or r is None:
        l, r = fixedPoints(A)

    # contract expectation value network
    o = ncon((l, r, A, A, np.conj(A), np.conj(A), O), ([6, 1], [5, 10], [1, 2, 3], [3, 4, 5], [6, 7, 8], [8, 9, 10], [2, 4, 7, 9]))

    return o

In [ ]:
def expVal2Mixed(O, Ac, Ar):
    """
    Calculate the expectation value of a 2-site operator in mixed gauge.

        Parameters
        ----------
        O : np.array(d, d, d, d)
            two-site operator,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauged.
        Ar : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right gauged.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    # contract expectation value network
    o = ncon((Ac, Ar, np.conj(Ac), np.conj(Ar), O), ([1, 2, 3], [3, 4, 5], [1, 6, 7], [7, 8, 5], [2, 4, 6, 8]), order=[3, 2, 4, 1, 6, 5, 8, 7])

    return o

In [ ]:
O2 = np.random.rand(d, d, d, d) + 1.0j * np.random.rand(d, d, d, d)

expVal = expVal2Uniform(O2, A)
expValGauge = expVal2Mixed(O2, Ac, Ar)
expValGauge2 = expVal2Mixed(O2, Al, Ac)

diff1 = abs(expVal - expValGauge)
diff2 = abs(expVal - expValGauge2)
assert diff1 < 1e-12 and diff2 < 1e-12, "different gauges give different values?"